In [1]:
# verified on 2/16/2025
# format of pass.txt
"""
google:your_email@gmail.com:your_google_app_password
yahoo:your_email@yahoo.com:your_yahoo_app_password
""";

In [2]:
import imaplib
import email
from email.header import decode_header
import pandas as pd
import re

# Yahoo IMAP settings
IMAP_SERVER = 'imap.gmail.com'
IMAP_PORT = 993
# EMAIL = 'your_email@gmail.com'
# PASSWORD = 'APP_password'

with open('pass.txt') as fin:
    strs=fin.readlines()
for s in strs:
    if s.split(":")[0]=='google':
        EMAIL=s.split(":")[1].strip()
        PASSWORD=s.split(":")[-1].strip()
# EMAIL,PASSWORD    

In [3]:
# Connect to the server
mail = imaplib.IMAP4_SSL(IMAP_SERVER, IMAP_PORT)
mail.login(EMAIL, PASSWORD)

MAILBOX='INBOX'

status, _ = mail.select(MAILBOX)
if status != 'OK':
    print(f"Failed to select mailbox: {MAILBOX}")

In [4]:
# status, validity_data = mail.status(MAILBOX, ['UIDVALIDITY'])
# uid_validity = validity_data[0].split()[2].decode('utf-8').strip(']')
# print(f"Current UIDVALIDITY: {uid_validity}")

In [5]:
start_date = '17-Feb-2025'
end_date = '18-Feb-2025'

search_criteria=f'(SINCE {start_date} BEFORE {end_date})'
search_criteria=f'(SINCE {start_date})'

# Search for emails within the date range
status, messages = mail.uid('search',None, search_criteria)
# status, messages = mail.search(None, f'SINCE "{start_date}"')
# status, messages = mail.search(None, 'ALL')

# sender_email = 'example@domain.com'
# search_criteria = f'(FROM "{sender_email}" SINCE "{start_date}" BEFORE "{end_date}")'

# Convert the message IDs to a list
target_uids = messages[0].split()

print(f"Found {len(target_uids)} emails.")


Found 10 emails.


In [6]:
results=[]
i=0
for uid in target_uids:
    i+=1
    # UID FETCH command
    status, data = mail.uid('fetch', uid, '(RFC822)')
    if status == 'OK' and data != [None]:
        r={}
        # Parse email content
        raw_email = data[0][1]
        msg = email.message_from_bytes(raw_email)
        sender=msg['From']
        print(f"\n{i} UID: {uid}")
        print(f"From: {sender}")
        
        # Decode headers
        subject, encoding = decode_header(msg['Subject'])[0]
        if isinstance(subject, bytes):
            try:
                subject = subject.decode(encoding if encoding else 'utf-8')
            except:
                subject="Decoding Error"        

        print(f"Subject: {subject}")
        
        r["id"]=uid
        r["sender"]=sender
        r["subject"]=subject
        results.append(r)           

    else:
        print(f"Email with UID {uid} not found")


1 UID: b'58644'
From: Evelyn Lee <wuxuexiaojie000@gmail.com>
Subject: 

2 UID: b'58645'
From: LinkedIn <jobs-listings@linkedin.com>
Subject: Senior Principal Signal Integrity Engineer role at Dell and other
 opportunities open

3 UID: b'58646'
From: LinkedIn Job Alerts <jobalerts-noreply@linkedin.com>
Subject: “senior hardware engineer”: Amaze Systems - Senior Digital Hardware Engineer and more

4 UID: b'58647'
From: LinkedIn <jobs-noreply@linkedin.com>
Subject: Yibing, apply to Senior Optical MSDV Hardware Engineer at NVIDIA
 and more

5 UID: b'58649'
From: LinkedIn <updates-noreply@linkedin.com>
Subject: Zhenxing (Jason) Wang posted: It was a highly productive conference with impressive progress on 1600ZR/ZR+ pr... 💡

6 UID: b'58652'
From: LinkedIn <notifications-noreply@linkedin.com>
Subject: Henry Duong and 4 others commented on your work anniversary

7 UID: b'58653'
From: LinkedIn <messages-noreply@linkedin.com>
Subject: Yibing, add Rommel Mercado - Product Safety Engineer

8 UID

In [7]:
def get_email(str_sender):
    pattern = re.compile(r'<(.*?)>')
    match = re.search(pattern, str_sender)

    if match:
        # print("Found:", match.group())
        email=match.group(1)
    else:
        # print("Not found")
        email="NA"   

    sender, encoding = decode_header(str_sender)[0]
    if isinstance(sender, bytes):
        sender = sender.decode(encoding if encoding else 'utf-8')

    return email,sender   

In [8]:
df=pd.DataFrame(results)
df[["email","sender"]]=df.apply(lambda r:get_email(r['sender']),axis=1,result_type='expand')

In [9]:
df.head()

,id,sender,subject,email
0,b'58644',Evelyn Lee <wuxuexiaojie000@gmail.com>,,wuxuexiaojie000@gmail.com
1,b'58645',LinkedIn <jobs-listings@linkedin.com>,Senior Principal Signal Integrity Engineer rol...,jobs-listings@linkedin.com
2,b'58646',LinkedIn Job Alerts <jobalerts-noreply@linkedi...,“senior hardware engineer”: Amaze Systems - Se...,jobalerts-noreply@linkedin.com
3,b'58647',LinkedIn <jobs-noreply@linkedin.com>,"Yibing, apply to Senior Optical MSDV Hardware ...",jobs-noreply@linkedin.com
4,b'58649',LinkedIn <updates-noreply@linkedin.com>,Zhenxing (Jason) Wang posted: It was a highly ...,updates-noreply@linkedin.com


In [10]:
dx=df.groupby("email").count().reset_index().sort_values(by="sender",ascending=True)
dx.tail()

,email,id,sender,subject
5,messages-noreply@linkedin.com,1,1,1
6,noreply@medium.com,1,1,1
7,notifications-noreply@linkedin.com,1,1,1
8,updates-noreply@linkedin.com,1,1,1
9,wuxuexiaojie000@gmail.com,1,1,1


In [11]:
list(dx.email.sort_values(ascending=False).values)

['wuxuexiaojie000@gmail.com',
 'updates-noreply@linkedin.com',
 'notifications-noreply@linkedin.com',
 'noreply@medium.com',
 'messages-noreply@linkedin.com',
 'jobs-noreply@linkedin.com',
 'jobs-listings@linkedin.com',
 'jobalerts-noreply@linkedin.com',
 'designideas@newsletters.edn.com',
 'IDX-noreply@google.com']

In [12]:
f1=['zmail@mail.zillow.com',
    'outreach.senate.ca.gov',
    'insideapple.apple.com',
    'medium.com',
    'newsletters.edn.com',
    'noreply@uber.com',
#  'yibingtang@meta.com',
#  'ybtang02@yahoo.com',
 'weekinreview@newsletters.eetimes.com',
 'webinar@newsletters.techonline.com',
 'vtgmail@list.vacationstogo.com',
#  'vimeo@vimeo.com',
 'vacationdeals@blgrnvacations.com',
 'updates@send-p.usnews.com',
#  'updates-noreply@linkedin.com',
#  'uber@uber.com',
 'ttsmarketplace@email.tiktok.com',
 'techpaper@newsletters.techonline.com',
 'survey@merewards.sg',
 'support@sc.mail.deepseek.com',
 'shelbysilva-silvacooptics.com@shared1.ccsend.com',
 'shein@usmail.shein.com',
#  'security@facebookmail.com',
#  'sam.glorit_x@etched.com',
#  'rineyprasanth@gmail.com',
 'redlobsteremail@em.redlobster.com',
#  'pge@email.opower.com',
 'open-houses@mail.zillow.com',
#  'notifications-noreply@linkedin.com',
#  'noreply@uber.com',
#  'noreply@em.pge.com',
#  'no_reply@post.applecard.apple',
#  'no_reply@email.apple.com',
#  'no-reply@em-s.dropbox.com',
#  'no-reply@accounts.google.com',
 'newsletters@e.consumeraffairs.com',
#  'newsletters-noreply@linkedin.com',
 'newsletter@email.kqed.org',
 'newsletter@divenewsletter.com',
 'mouser@info.techonline-direct.com',
#  'messaging-digest-noreply@linkedin.com',
#  'messages-noreply@linkedin.com',
 'members@m.consumeraffairs.com',
 'members@e.consumeraffairs.com',
 'member@email.kqed.org',
 'marketing@thesamragroup.ccsend.com',
 'mail@mail.adobe.com',
 'mail@eg.hotels.com',
 'mail@eg.expedia.com',
#  'magloirevishaal@gmail.com',
#  'linkedin@e.linkedin.com',
#  'jobs-noreply@linkedin.com',
#  'jobs-listings@linkedin.com',
#  'jobalerts-noreply@linkedin.com',
#  'invitations@linkedin.com',
 'instant-updates@mail.zillow.com',
#  'inmail-hit-reply@linkedin.com',
 'info@e.ubm-info.net',
 'iheartradio@listen.iheart.com',
#  'icloudplus@insideapple.apple.com',
#  'hit-reply@linkedin.com',
 'hello@mkt.getyourguide.com',
 'hello@mail.grammarly.com',
#  'groups-noreply@linkedin.com',
 'funfriday@newsletters.edn.com',
 'email@market.temuemail.com',
#  'editors-noreply@linkedin.com',
#  'developer@insideapple.apple.com',
 'delights@enews.merewards.sg',
 'club@castellodiamorosa.com',
 'cameronhart@silvacooptics.ccsend.com',
 'audiexperience@e.audiusa.com',
#  'appstore@insideapple.apple.com',
#  'appletv@insideapple.apple.com',
#  'applepay@insideapple.apple.com',
#  'applenewsplus@insideapple.apple.com',
#  'applemusic@insideapple.apple.com',
#  'applefitnessplus@insideapple.apple.com',
#  'applearcade@insideapple.apple.com',
 'SouthwestAirlines@iluv.southwest.com',
#  'RepRoKhanna@mail.house.gov',
#  'RapidRewards@pts.southwest.com',
#  'PGECustomerService@notifications.pge.com',
 'Nixplay@marketing.nixplay.com',
#  'NA',
#  'DoNotReply@qooway.com',
#  'Costco@digital.costco.com'
 ]

In [13]:
dx[dx.subject>3].email.values

array([], dtype=object)

In [14]:
email_to_delete=f1+['blog.secretprofittools.com','pro.milkymoneyway.com','ldii.longdistanceinvestings.com','first.AmericanTruthCast.com','today.investingwithmind.com','day.usastocksinsider.com',"best.marketexpopro.com",'net.richpeoplenetworks.com','fin.finacetechfuture.com','editor@nowcc.notesonwealthcreation.com','prisintol.com','tfm.thefinancememories.com','today.investingwithmind.com','ucc.urbancapitalist.com','digest.marketbroadview.com','tips.sportinvestorsleague.com','cottonbubblelay.com','deals@deals.vegas.com','find@via.healthyinvestingtips.com','webinar@newsletters.techonline.com','l.barnett@wtf.wealthtimefuture.com','get@real.multitradersmoves.com','mailing@ceg.certaineconomicgrowth.com','temu@updates.temuofficial.com','strategy@insights.FinancesDataDriven.com','ruffus.white@net.richpeoplenetworks.com','with@top.goldmineadvisors.com','your@very.smartsuccessstrategy.com','news@daily.investmentjourneys.com','check@tips.sportinvestorsleague.com','daily@digest.marketbroadview.com','q.hayes@choice.besttoinvestnews.com','news@daily.financiffy.com','daily@blog.secretprofittools.com','mail@your.usagoldmiracle.com','the.directionforexpo.com','daily@digest.marketbroadview.com']

In [15]:
subject_to_delete =['David Kant','FunWords Team','richard grant','stockpile saving','satelite earnings now','this token could','David Kant','Nextdish','hotels.com', 'Matthew Wehner','Elizabeth Harris', 'Trump','David Kant','Part-Time Work','Investors League','MileagePlus Partner','Russia in Shock','woot.com','Christopher Johnson','Tax-Free Cash Hack']

In [16]:
pattern1 = '|'.join(subject_to_delete)
pattern2 = '|'.join(email_to_delete)

In [17]:

dz=df[(df.sender.str.contains(pattern1, case=False)) | (df.email.str.contains(pattern2, case=False))]

In [18]:
dz

,id,sender,subject,email
8,b'58655',EDN Design Ideas <designideas@newsletters.edn....,"Tek 576 curve tracer adapters, Flip ON Flop OF...",designideas@newsletters.edn.com
9,b'58656',Medium Daily Digest <noreply@medium.com>,What’s,noreply@medium.com


In [19]:
uids_to_delete=list(dz.id.unique())
uids_to_delete, len(uids_to_delete)

([b'58655', b'58656'], 2)

In [20]:
# Delete emails
i=0
for uid in uids_to_delete:
    i+=1
    print(i,uid)
    try:
        status, response = mail.uid('store', uid, '+FLAGS', '\\Deleted')
        print(f"Delete status: {status}, Response: {response}")
    except:
        print(uid,"not working")
        
# Expunge deleted emails
mail.expunge()

1 b'58655'
Delete status: OK, Response: [b'40612 (FLAGS (\\Seen \\Deleted) UID 58655)']
2 b'58656'
Delete status: OK, Response: [b'40612 (FLAGS (\\Seen \\Deleted) UID 58656)']


('OK', [b'40612', b'40612'])

In [21]:

# Close connection
mail.close()
mail.logout()

('BYE', [b'LOGOUT Requested'])